In [16]:
"""
使用基本的特征构建基本的 xgboost 模型
@author: MarkLiu
@time  : 17-5-25 下午9:03
"""
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import numpy as np
import pandas as pd
import xgboost as xgb
# remove warnings
import warnings

warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

# my own module
from features import data_utils

In [17]:
def CV(isLog1p=True):
    train, test, macro = data_utils.load_data()

    mult = .969

    train['price_doc'] = train["price_doc"] * mult + 10
    if (isLog1p):
        train['price_doc'] = np.log1p(train['price_doc'])
    ylog_train_all = train['price_doc']
    id_train = train['id']
    train.drop(['id', 'price_doc'], axis=1, inplace=True)
    #submit_ids = test['id']
    submit_ids = pd.read_csv('../../input/test.csv')['id']
    test.drop(['id'], axis=1, inplace=True)

    # 合并训练集和测试集
    conbined_data = pd.concat([train[test.columns.values], test])
    # macro_cols = ["balance_trade", "balance_trade_growth", "eurrub", "average_provision_of_build_contract",
    #               "micex_rgbi_tr", "micex_cbi_tr", "deposits_rate", "mortgage_value", "mortgage_rate",
    #               "income_per_cap", "rent_price_4+room_bus", "museum_visitis_per_100_cap", "apartment_build", "timestamp"]
    # conbined_data = pd.merge_ordered(conbined_data, macro[macro_cols], on='timestamp', how='left')

    conbined_data.drop(['timestamp'], axis=1, inplace=True)
    print "conbined_data:", conbined_data.shape

    # Deal with categorical values
    for c in conbined_data.columns:
        if conbined_data[c].dtype == 'object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(conbined_data[c].values))
            conbined_data[c] = lbl.transform(list(conbined_data[c].values))

    train = conbined_data.iloc[:train.shape[0], :]
    test = conbined_data.iloc[train.shape[0]:, :]

    test_size = (1.0 * test.shape[0]) / train.shape[0]
    print "submit test size:", test_size

    # Convert to numpy values
    X_all = train.values

    # Create a validation set, with last 20% of data
    num_train = int(train.shape[0] / (1+test_size))

    X_train_all = X_all
    X_test = test

    # X_train = X_all[:num_train]
    # X_val = X_all[num_train:]
    # ylog_train = ylog_train_all[:num_train]
    # ylog_val = ylog_train_all[num_train:]

    X_train, X_val, ylog_train, ylog_val=train_test_split(X_all, ylog_train_all, test_size=test_size, 
                                                random_state=123)

    print "validate size:", 1.0*X_val.shape[0] / X_train.shape[0]

    df_columns = train.columns

    print('X_train_all shape is', X_train_all.shape)
    print('X_train shape is', X_train.shape)
    print('y_train shape is', ylog_train.shape)
    print('X_val shape is', X_val.shape)
    print('y_val shape is', ylog_val.shape)
    print('X_test shape is', X_test.shape)
    
    dtrain_all = xgb.DMatrix(X_train_all, ylog_train_all, feature_names=df_columns)
    dtrain = xgb.DMatrix(X_train, ylog_train, feature_names=df_columns)
    dval = xgb.DMatrix(X_val, ylog_val, feature_names=df_columns)
    dtest = xgb.DMatrix(X_test, feature_names=df_columns)

    xgb_params = {
        'eta': 0.05,
        'max_depth': 5,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'silent': 1,
        'seed':5
    }

    num_round = 1000
    xgb_params['nthread'] = 24
    evallist = [(dval, 'eval')]

    bst = xgb.train(xgb_params, dtrain, num_round, evallist, early_stopping_rounds=40, verbose_eval=10)
    
    if (isLog1p):
        train_rmse = mean_squared_error(ylog_train, bst.predict(dtrain))
        val_rmse = mean_squared_error(ylog_val, bst.predict(dval))
    else:
        train_rmse = mean_squared_error(np.log1p(ylog_train), np.log1p(bst.predict(dtrain)))
        val_rmse = mean_squared_error(np.log1p(ylog_val), np.log1p(bst.predict(dval)))
    print 'train_rmse =', np.sqrt(train_rmse), ', val_rmse =', np.sqrt(val_rmse)

In [18]:
CV(True)
#not log1p, original train_rmse = 0.252746403304 , val_rmse = 0.320226748185
#log1p, orginal train_rmse = 0.257648786803 , val_rmse = 0.320517933831, 

#log1p, my deleted train_rmse = 0.257648786803 , val_rmse = 0.320517933831, 
#log1p, my deleted added train_rmse = 0.257648786803 , val_rmse = 0.320517933831

conbined_data: (36650, 702)
submit test size: 0.264316268801
validate size: 0.359279752415
('X_train_all shape is', (28988, 702))
('X_train shape is', (21326, 702))
('y_train shape is', (21326,))
('X_val shape is', (7662, 702))
('y_val shape is', (7662,))
('X_test shape is', (7662, 702))
[0]	eval-rmse:14.401
Will train until eval-rmse hasn't improved in 40 rounds.
[10]	eval-rmse:8.62748
[20]	eval-rmse:5.17376
[30]	eval-rmse:3.11093
[40]	eval-rmse:1.88354
[50]	eval-rmse:1.1597
[60]	eval-rmse:0.744146
[70]	eval-rmse:0.517844
[80]	eval-rmse:0.406038
[90]	eval-rmse:0.356851
[100]	eval-rmse:0.336461
[110]	eval-rmse:0.328658
[120]	eval-rmse:0.325175
[130]	eval-rmse:0.323587
[140]	eval-rmse:0.322657
[150]	eval-rmse:0.322152
[160]	eval-rmse:0.321778
[170]	eval-rmse:0.32146
[180]	eval-rmse:0.321216
[190]	eval-rmse:0.320811
[200]	eval-rmse:0.320791
[210]	eval-rmse:0.320634
[220]	eval-rmse:0.320532
[230]	eval-rmse:0.320243
[240]	eval-rmse:0.320344
[250]	eval-rmse:0.320179
[260]	eval-rmse:0.320107